In [290]:
import csv
import numpy as np
import torch
import json
import vegas
from binNF.normalizing_flows.manager import *
import math
torch.set_default_dtype(torch.double)

def create_fun(gn, gw):
    
    if(gn==1):
        def f(x):
                return torch.exp(-torch.sum((x-0.5)**2/(gw**2),-1))
        return f
    
    if(gn==2):
        def f(x):
                return torch.exp(-torch.sum((x-0.25)**2/(gw**2),-1))+torch.exp(-torch.sum((x-0.75)**2/(gw**2),-1))
        return f
    
    if(gn==4):
        def f(x):
            shift=torch.ones_like(x)*0.25
            shift1=shift.clone()*3
            lim=int((shift.shape[1]/2))
            shift2=torch.cat((shift[:,:lim],shift1[:,lim:]),-1)
            shift3=torch.cat((shift1[:,:lim],shift[:,lim:]),-1)
            return torch.exp(-torch.sum((x-shift)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift1)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift2)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift3)**2/(gw**2),-1))
        return f
    
    if(gn==8):
        def f(x):
            shift=torch.ones_like(x)*0.25#000
            shift1=shift.clone()*3#111
            lim=int((shift.shape[1]/3))
            shift2=torch.cat((shift[:,:lim],shift1[:,lim:2*lim],shift[:,2*lim:]),-1) #010
            shift3=torch.cat((shift1[:,:lim],shift[:,lim:]),-1)#100
            shift4=torch.cat((shift1[:,:lim],shift1[:,lim:2*lim],shift[:,2*lim:]),-1) #110
            shift5=torch.cat((shift[:,:2*lim],shift1[:,2*lim:]),-1) #001
            shift6=torch.cat((shift1[:,:lim],shift[:,lim:2*lim],shift1[:,2*lim:]),-1) #101
            shift7=torch.cat((shift[:,:lim],shift1[:,lim:]),-1) #011
            return torch.exp(-torch.sum((x-shift)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift1)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift2)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift3)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift4)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift5)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift6)**2/(gw**2),-1))+torch.exp(-torch.sum((x-shift7)**2/(gw**2),-1))
        return f
def create_funv(gn, gw):
    
    if(gn==1):
        def f(x):
                return np.exp(-np.sum((x-0.5)**2/(gw**2),-1))
        return f # peak at [0.5,...,0.5]
    
    if(gn==2):
        def f(x):
                return np.exp(-np.sum((x-0.25)**2/(gw**2),-1))+np.exp(-np.sum((x-0.75)**2/(gw**2),-1))
        return f  #peak at [0.25,...,0.25] and [0.75,...,0.75]
    
    if(gn==4):
        
        def f(x):
            shift=np.ones_like(x)*0.25
            shift1=shift.copy()*3
            lim=int((shift.shape[0]/2))
            shift2=np.concatenate((shift[:lim],shift1[lim:]),-1)
            shift3=np.concatenate((shift1[:lim],shift[lim:]),-1)
            return np.exp(-np.sum((x-shift)**2/(gw**2),-1))+np.exp(-np.sum((x-shift1)**2/(gw**2),-1))+np.exp(-np.sum((x-shift2)**2/(gw**2),-1))+np.exp(-np.sum((x-shift3)**2/(gw**2),-1))
        return f #peak at [0.25,...,0.25], [0.75,...,0.75],[0.25,...,0.75], [0.25,...,0.75]
                #in dim2, there would be in the [0,1] plane only two peaks, as two are identical
                # other than gn=2, the [1,2] plane has 4 peaks
              
    
    if(gn==8):
        
        def f(x):
            shift=np.ones_like(x)*0.25#000
            shift1=shift.copy()*3#111
            lim=int(shift.shape[0]/3)
            shift2=np.concatenate((shift[:lim],shift1[lim:2*lim],shift[2*lim:]),0) #010
            shift3=np.concatenate((shift1[:lim],shift[lim:]),0)#100
            shift4=np.concatenate((shift1[:lim],shift1[lim:2*lim],shift[2*lim:]),0) #110
            shift5=np.concatenate((shift[:2*lim],shift1[2*lim:]),0) #001
            shift6=np.concatenate((shift1[:lim],shift[lim:2*lim],shift1[2*lim:]),0) #101
            shift7=np.concatenate((shift[:lim],shift1[lim:]),0) #011
            return np.exp(-np.sum((x-shift)**2/(gw**2),-1))+np.exp(-np.sum((x-shift1)**2/(gw**2),-1))+np.exp(-np.sum((x-shift2)**2/(gw**2),-1))+np.exp(-np.sum((x-shift3)**2/(gw**2),-1))+ np.exp(-np.sum((x-shift4)**2/(gw**2),-1))+np.exp(-np.sum((x-shift5)**2/(gw**2),-1))+np.exp(-np.sum((x-shift6)**2/(gw**2),-1))+np.exp(-np.sum((x-shift7)**2/(gw**2),-1))
        return f 
    
with open('/home/niklas/Desktop/Thesis_code/Storage_runs/mfrun_full/list_c2', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    lister=[]
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        lister.append(row)
        #print(f'\t{row["name"]} works in the {row["department"]} department, and was born in {row["birthday month"]}.')
        line_count += 1
    print(f'Processed {line_count} lines.')
    #print(container.items())
    for nps in [1]:  #!!!!
        for wp in [0.15,0.25,0.35]:
            for dim in [2]:
                f=create_fun(nps,wp)
                fv=create_funv(nps,wp)
                w = torch.empty(500000,dim)
                torch.nn.init.uniform_(w)
                f_res=f(w)
                funcmean_sq=(torch.mean(f_res)**2).numpy() 
                fvar=torch.var(f_res).numpy()
                #print("Original variance for {0:d} peaks with width {1:2f}: {2:5E}".format(np,wp,fvar))
                #print(funcmean_sq)
                for modus in [' NIS',' VEGAS']:
                    if(modus==" NIS"):
                        minavg=1000
                        minstd=1000
                        min_nnl=0
                        min_nnw=0
                        integavg=0
                        integerr=0
                        meancount=0
                        stdcount=0
                        meancv=0
                        stdcv=0
                        for nnl in [3,8]:
                            for nnw in [6,11,16]:
                                for c in [int(2*np.ceil(npy.log2(dim))), int(4*npy.ceil(np.log2(dim)))]:
                                    


                                    x=[v for  v in lister if (int(v[" number of peaks"]) == nps and v[" modus"]==modus and 
                                                              float(v[" width of peaks"]) == wp and int(v["dimensions"])==dim and int(v[" NN_length"])==nnl and int(v[" NN_width"])==nnw)]
                                    ider=(npy.array([l["ID"] for l in x]).astype(npy.int))
                                    counter=0
                                    for i in range(len(ider)):
                                        with open('/home/niklas/Desktop/Thesis_code/Storage_runs/mfrun_full/NIS/'+str(ider[i])+'/config.json', 'r') as myfile:
                                            data=myfile.read()

                                            obj = json.loads(data)
                                           
                                            
                                            if(obj['n_cells']!=c):
                                                x.remove(x[i-counter])
                                                counter+=1
                                                continue
                                            n_bins=(obj['n_bins'])
                                            
                                        
                                       
                                    ider=(np.array([l["ID"] for l in x]).astype(np.int))
                                    z=(np.array([l[" best_loss"] for l in x]).astype(np.double)-funcmean_sq)/fvar
                                    y=(npy.array([l[" best_count"] for l in x]).astype(npy.double))
                                    mean=np.mean(z)
                                    meanc=np.mean(y)
                                    var=np.var(z)
                                    std=np.std(z,ddof=1)/np.sqrt(len(z))
                                    stdc=np.std(y,ddof=1)/np.sqrt(len(z))

                                    n_flow=dim
                                    NF =  PWQuadManager(n_flow=n_flow)
                                    NF.create_model(n_cells=c,n_bins=n_bins, NN=[nnw]*nnl,dev=torch.device("cpu"))
                                    
                                    integl=[]
                                    errl=[]
                                    for i in range(len(ider)):
                                        model=torch.load('/home/niklas/Desktop/Thesis_code/Storage_runs/mfrun_full/NIS/'+str(ider[i])+'/torch',map_location=torch.device('cpu'))
                                        #print(ider[i])
                                        NF.best_model.load_state_dict(model['model_state_dict'])
                                        w = torch.empty((10000,n_flow)) 
                                        torch.nn.init.uniform_(w,0,1)
                                        Y=NF.format_input(w, dev=torch.device("cpu"))
                                        XZ=NF.best_model(Y)
                                        integl.append(torch.mean(f(Y[:,:-1])*Y[:,-1]))
                                        errl.append(torch.std(f(Y[:,:-1])*Y[:,-1])/math.sqrt(10000))
                                       


                                    #square added is over the runs!
                                    if(mean<minavg and (z == 1).sum()<=2 and (z==npy.nan).sum()<=2):
                                        minavg=mean
                                        minstd=std
                                        min_nnl=nnl
                                        min_nnw=nnw
                                        meancount=meanc
                                        stdcount=stdc
                                        integavg=np.mean(integl)
                                        integerr=np.sum(npy.power(errl,2))
                                    
                        
                    else:
                       # print(modus)
                        x=[v for  v in lister if (int(v[" number of peaks"]) == nps and v[" modus"]==modus and 
                                                          float(v[" width of peaks"]) == wp and int(v["dimensions"])==dim)]
                        #print(x)
                        z=(npy.array([l[" best_loss"] for l in x]).astype(npy.double)-funcmean_sq)/fvar
                        y=(npy.array([l[" best_count"] for l in x]).astype(npy.double))
                        meancv=npy.mean(y)
                        
                        stdcv=np.std(y,ddof=1)/np.sqrt(len(z))
                       # print(z)
                        meanv=np.mean(z)
                        #varv=npy.var(z)
                        stdv=npy.std(z,ddof=1)/np.sqrt(len(z))
                        integ = vegas.Integrator([[0, 1]*dim])
                        result=integ(fv, nitn=10, neval=1000)
                        print("VEGAS")
                        print(result.mean)
                        print(result.sdev)
                        print("###")
                        
                
                impratio=minavg/meanv
                #print(impratio)
                preprec2=int(np.ceil(np.log10(minavg)))
                preprec1=int(np.ceil(np.log10(meanv)))
                prec2=int(-np.ceil(np.log10(minstd))+2+preprec2)
                prec1=int(-np.ceil(np.log10(stdv))+2+preprec1)
                           
                ppreprec2=int(np.ceil(np.log10(meancount)))
                ppreprec1=int(np.ceil(np.log10(meancv)))
                pprec2=int(-np.ceil(np.log10(stdcount))+2+ppreprec2)
                if (stdcv==0):
                    pprec=0
                else:
                    pprec1=int(-np.ceil(np.log10(stdcv))+2+ppreprec1)
        
                #print(prec1)
                #print(prec2)
                print("NIS")
                print(integavg)
                print(integerr)
                
                if(minavg<1):
                    if(wp<0.3):
                        print("& \\textbf{{{0:.2f}}}    & {6:.3E}     &  {1:.{prec}E} \\pm {2:.2E} &  {7:.{pprec}E} \\pm {8:.2E}    &   {3:.{prec1}E} \\pm {4:.2E}   &   {9:.{pprec1}E} \\pm {10:.2E} &  {5:.3E}   \\\\ \cline{{2-8}}"
                              .format(wp,minavg, minstd, meanv, stdv, impratio, fvar,meancount, stdcount,meancv,stdcv,pprec=pprec2,prec=prec2,prec1=prec1,pprec1=pprec1))
                    else:
                         print("& \\textbf{{{0:.2f}}}    & {6:.3E}       &  {1:.{prec}E} \\pm {2:.2E}   &  {7:.{pprec}E} \\pm {8:.2E}   &   {3:.{prec1}E} \\pm {4:.2E}  &   {9:.{pprec1}E} \\pm {10:.2E}&  {5:.3E}   \\\\ \hline"
                               .format(wp,minavg, minstd, meanv, stdv, impratio,fvar,meancount, stdcount,meancv,stdcv,pprec=pprec2, prec=prec2,prec1=prec1,pprec1=pprec1))
                else:
                    if(wp<0.3):
                        print("& \\textbf{{{0:.2f}}}    & {6:.3E}       &  ---  &  ---      &   {3:.{prec1}E} \\pm {4:.2E}  &   {7:.{pprec1}E} \\pm {8:.2E} & ---     \\\\ \cline{{2-8}}".format(wp,minavg, minstd, meanv, stdv,meancv,stdcv, fvar,prec=prec2,prec1=prec1,pprec1=pprec1))
                    else:
                         print("& \\textbf{{{0:.2f}}}    & {6:.3E}       &  ---  &  ---     &   {3:.{prec1}E} \\pm {4:.2E}  &   {7:.{pprec1}E} \\pm {8:.2E} & ---     \\\\ \hline".format(wp,minavg, minstd, meanv, stdv,fvar,meancv,stdcv,prec=prec2,prec1=prec1,pprec1=pprec1))
                

Column names are ID,  number of peaks,  width of peaks,  NN_length,  NN_width, dimensions,  modus,  best_loss,  best_loss_rel, var_loss,  best_epoch,  best_count,  time,  time_total
Processed 3256 lines.
VEGAS
0.265849955719651
3.331307314705113e-05
###
NIS
0.07057665441682512
1.2192563768493833e-05
& \textbf{0.15}    & 3.024E-02     &  2.975E-01 \pm 4.87E-02 &  1.810E+06 \pm 4.33E+05    &   1.6862E-02 \pm 1.31E-04   &   2.1567E+05 \pm 3.54E+03 &  1.765E+01   \\ \cline{2-8}
VEGAS
0.4408631334740892
4.108244493123405e-05
###
NIS
0.19446677508716398
2.427350498867546e-05
& \textbf{0.25}    & 6.063E-02     &  3.585E-01 \pm 4.82E-02 &  4.320E+06 \pm 6.54E+05    &   1.3360E-02 \pm 2.81E-04   &   3.0759E+05 \pm 5.54E+03 &  2.683E+01   \\ \cline{2-8}
VEGAS
0.5935260738553
3.6424873440597466e-05
###
NIS
0.35301585715380224
2.6745193455976176e-05
& \textbf{0.35}    & 6.672E-02       &  3.570E-01 \pm 1.34E-02   &  1.8100E+07 \pm 8.16E+05   &   9.594E-03 \pm 3.97E-04  &   3.924E+05 \pm 1.18E+04& 